# Deeplabv3 + LIP

## Path definitions - change to your own

In [ ]:
ROOT_PROJECT = "/content/drive/MyDrive/TFM"
TRAIN_VAL_IMAGES = os.path.join(ROOT_PROJECT, 'lip_trainval_images')
TRAIN_VAL_SEGMENTATIONS = os.path.join(ROOT_PROJECT, 'lip_trainval_segmentations')
TRAIN_TFRECORD = os.path.join(ROOT_PROJECT, 'train_lip_tfrecord')
VAL_TFRECORD = os.path.join(ROOT_PROJECT, 'val_lip_tfrecord')
TRAIN_VAL_TFRECORD = os.path.join(ROOT_PROJECT, 'trainval_lip_tfrecord')
CHECKPOINT = os.path.join(ROOT_PROJECT, 'checkpoint_lip_mobilenet')
PRETRAINED_MODEL = os.path.join(ROOT_PROJECT, 'deeplabv3_mnv2_pascal_train_aug/model.ckpt-30000')
EVAL_RESULTS = os.path.join(ROOT_PROJECT, 'eval_results_lip')
VIS_RESULTS = os.path.join(ROOT_PROJECT, 'vis_results_lip')
EXPORT = os.path.join(ROOT_PROJECT, 'exported_model')

## Setup Deeplabv3

In [ ]:
!pip install tensorflow-gpu==1.15.3
!pip install tf_slim==1.0.0

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3

Here Google Drive storage is loaded. You can omit this cell if you are not using it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd {ROOT_PROJECT}

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
%cd {ROOT_PROJECT}/models/research/

In [ ]:
%env PYTHONPATH={ROOT_PROJECT}/models/research/:{ROOT_PROJECT}/models/research/deeplab:{ROOT_PROJECT}/models/research/slim:/env/python

In [ ]:
%cd {ROOT_PROJECT}

## Using LIP

### Unzip the dataset

In [ ]:
!unzip {ROOT_PROJECT}/LIP/TrainVal_images.zip -d {TRAIN_VAL_IMAGES}

In [ ]:
!unzip -q {TRAIN_VAL_IMAGES}/TrainVal_images.zip -d {TRAIN_VAL_IMAGES}/TrainVal_images

In [ ]:
!unzip {ROOT_PROJECT}/LIP/TrainVal_parsing_annotations.zip -d {TRAIN_VAL_SEGMENTATIONS}

In [ ]:
!unzip -q {TRAIN_VAL_SEGMENTATIONS}/TrainVal_parsing_annotations/TrainVal_parsing_annotations.zip -d {TRAIN_VAL_SEGMENTATIONS}/TrainVal_parsing_annotations/

### Build tfrecord of the dataset

In [ ]:
!mkdir {TRAIN_TFRECORD}
!mkdir {VAL_TFRECORD}

In [ ]:
!python models/research/deeplab/datasets/build_voc2012_data.py \
  --image_folder="{TRAIN_VAL_IMAGES}/TrainVal_images/train_images" \
  --semantic_segmentation_folder="{TRAIN_VAL_SEGMENTATIONS}/TrainVal_parsing_annotations/train_segmentations" \
  --list_folder="{TRAIN_VAL_IMAGES}" \
  --image_format="jpg" \
  --output_dir="{TRAIN_TFRECORD}"

In [ ]:
!python models/research/deeplab/datasets/build_voc2012_data.py \
  --image_folder="{TRAIN_VAL_IMAGES}/TrainVal_images/val_images" \
  --semantic_segmentation_folder="{TRAIN_VAL_SEGMENTATIONS}/TrainVal_parsing_annotations/val_segmentations" \
  --list_folder="{TRAIN_VAL_IMAGES}" \
  --image_format="jpg" \
  --output_dir="{VAL_TFRECORD}"

#### Move both tfrecord to a single directory and rename them

For simplicity reasons, **rename** command is used. However, **os.rename** could be used instead

In [ ]:
%cd {TRAIN_TFRECORD}

In [ ]:
!rename 's/_id//;' *

In [ ]:
%cd {VAL_TFRECORD}

In [ ]:
!rename 's/_id//;' *

In [ ]:
%mv {VAL_TFRECORD}/* {TRAIN_VAL_TFRECORD}
%mv {TRAIN_TFRECORD}/* {TRAIN_VAL_TFRECORD}

## Train with LIP
Use the --dataset param with lip or cihp to change the training dataset

In [ ]:
%mkdir {CHECKPOINT}

In [ ]:
%cd {ROOT_PROJECT}/models/research/

In [ ]:
!python deeplab/model_test.py

Use the following parameter only if its the first time you run it: --tf_initial_checkpoint="{PRETRAINED_MODEL}" \

In [ ]:
!python deeplab/train.py --logtostderr \
  --training_number_of_steps=60000 \
  --train_split="train" \
  --model_variant="mobilenet_v2" \
  --atrous_rates=6 \
  --atrous_rates=12 \
   --atrous_rates=18 \
   --output_stride=16 \
   --decoder_output_stride=4 \
   --train_batch_size=1 \
   --dataset="lip" \
   --train_logdir="{CHECKPOINT}" \
   --dataset_dir="{TRAIN_VAL_TFRECORD}" \
   --fine_tune_batch_norm=false \
   --initialize_last_layer=false \
   --last_layers_contain_logits_only=false

## Evaluation

In [ ]:
%mkdir {EVAL_RESULTS}

In [ ]:
!python deeplab/eval.py --logtostderr \
  --eval_split="val" \
  --model_variant="mobilenet_v2" \
  --eval_crop_size="1080,1080" \
  --atrous_rates=6 \
  --atrous_rates=12 \
   --atrous_rates=18 \
   --output_stride=16 \
   --decoder_output_stride=4 \
   --dataset="lip" \
   --checkpoint_dir="{CHECKPOINT}" \
   --eval_logdir="{EVAL_RESULTS}" \
   --dataset_dir="{TRAIN_VAL_TFRECORD}" \
   --max_number_of_iterations=1 \
   --eval_interval_secs=0

## Visualization

In [ ]:
%mkdir {VIS_RESULTS}

In [ ]:
!python deeplab/vis.py --logtostderr \
  --vis_split="val" \
  --model_variant="mobilenet_v2" \
  --vis_crop_size="1080,1080" \
  --atrous_rates=6 \
  --atrous_rates=12 \
   --atrous_rates=18 \
   --output_stride=16 \
   --decoder_output_stride=4 \
   --dataset="lip" \
   --checkpoint_dir="{CHECKPOINT}" \
   --vis_logdir="{VIS_RESULTS}" \
   --dataset_dir="{TRAIN_VAL_TFRECORD}" \
   --max_number_of_iterations=1 \
   --eval_interval_secs=0

## Export trained model

In [ ]:
%mkdir {EXPORT}

In [ ]:
!python deeplab/export_model.py \
    --logtostderr \
    --checkpoint_path="{CHECKPOINT}/model.ckpt-60000" \
    --export_path="{EXPORT}/frozen_inference_graph.pb"  \
    --model_variant="mobilenet_v2"  \
    --dataset="lip" \
    --atrous_rates=6  \
    --atrous_rates=12  \
    --atrous_rates=18   \
    --output_stride=16  \
    --crop_size=1080 \
    --crop_size=1080 \
    --decoder_output_stride=4  \
    --num_classes=19 \
    --inference_scales=1.0